<a href="https://colab.research.google.com/github/UttkarshM/collaborative_filtering/blob/master/Collaborative_Filtering_Movies_kaggleipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Recommendation systems


Recommendation systems are usually based on two types of systems:
1. content based filter
2. collaborative based filter

content based filter recommends you things based on your previous preferences<br><br>
whereas collaborative based filter recommends you things based on the what similar users liked
(the things u liked in the past are used to get the similarity between u and the user).

## Cosine Similarity


Cosine similarity measures the similarity between two vectors of an inner product space. It is measured by the cosine of the angle between two vectors and determines whether two vectors are pointing in roughly the same direction. It is often used to measure document similarity in text analysis.

links:
<br>movie recommendation systems<br>
https://www.freecodecamp.org/news/how-to-build-a-movie-recommendation-system-based-on-collaborative-filtering/

In [1]:
!pip install fuzzywuzzy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
var =  'drive/MyDrive/Datasets/DataSets/'

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

#TODO <br>
<b>clean the data to make it more accurate and recent.<b>

In [6]:
user_ratings_df = pd.read_csv(var+'/ratings.csv')
user_ratings_df

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
...,...,...,...,...
26024284,270896,58559,5.0,1257031564
26024285,270896,60069,5.0,1257032032
26024286,270896,63082,4.5,1257031764
26024287,270896,64957,4.5,1257033990


In [7]:
movie_data = pd.read_csv(var+"/movies_metadata.csv")
movie_metadata = movie_data[["title","genres"]]
# movie_metadata.iloc[0]["genres"]
movie_metadata = movie_metadata[:40000]

<ipython-input-7-146212a87e86>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_data = pd.read_csv(var+"/movies_metadata.csv")


In [8]:
tester = pd.read_csv(var+"/ratings.csv")
tester

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
...,...,...,...,...
26024284,270896,58559,5.0,1257031564
26024285,270896,60069,5.0,1257032032
26024286,270896,63082,4.5,1257031764
26024287,270896,64957,4.5,1257033990


user_ratings_df.columns
user_ratings_df = pd.concat([user_ratings_df,])

In [9]:
movie_data = pd.concat([user_ratings_df,movie_metadata],axis=1).fillna(0)
# movie_data = movie_data.iloc[:40000]

In [10]:
movie_data

,userId,movieId,rating,timestamp,title,genres
0,1,110,1.0,1425941529,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,1,147,4.5,1425942435,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,1,858,5.0,1425941523,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,1,1221,5.0,1425941546,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,1,1246,5.0,1425941556,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]"
...,...,...,...,...,...,...
26024284,270896,58559,5.0,1257031564,0,0
26024285,270896,60069,5.0,1257032032,0,0
26024286,270896,63082,4.5,1257031764,0,0
26024287,270896,64957,4.5,1257033990,0,0


In [11]:
movie_titles = movie_data[["title"]]
movie_titles.to_numpy()

array([['Toy Story'],
       ['Jumanji'],
       ['Grumpier Old Men'],
       ...,
       [0],
       [0],
       [0]], dtype=object)

In [12]:
for i in movie_metadata.iloc[0]:
  print(i)

Toy Story
[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]


user_ratings_df.columns,user_ratings_df.head()

movie_data = movie_metadata.merge(user_ratings_df,on="movieId")
movie_data.head()

In [13]:
user_ratings_df = user_ratings_df.iloc[:40000]

In [102]:
user_item_matrix = user_ratings_df.pivot(index=["userId"],columns=["movieId"],values='rating').fillna(0)
user_item_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,168174,168252,168366,168434,168612,170355,170705,171763,172547,174585
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
429,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
430,3.0,0.0,3.0,0.0,3.0,5.0,5.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
movie_data = movie_data[:40000]

In [104]:
movie_data = movie_data.dropna(0)

<ipython-input-104-45f7f092fe56>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  movie_data = movie_data.dropna(0)


In [105]:
user_item_matrix.shape

(432, 5719)

In [106]:
cf_knn_model = NearestNeighbors(metric="cosine",algorithm="brute",n_neighbors=10,n_jobs=-1)
cf_knn_model.fit(user_item_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

data manipulation

In [107]:
# it = pd.concat([user_item_matrix,user],axis=0).fillna(0)
# u1 = pd.DataFrame(np.zeros(5719).reshape(1,-1))
cols= user_item_matrix.columns
# user_item_matrix.append(,ignore_index=True)
# user_item_matrix.shape
cols

Int64Index([     1,      2,      3,      4,      5,      6,      7,      8,
                 9,     10,
            ...
            168174, 168252, 168366, 168434, 168612, 170355, 170705, 171763,
            172547, 174585],
           dtype='int64', name='movieId', length=5719)

In [130]:
temp = user_item_matrix
arr = temp.to_numpy()
# print(arr.shape)
tempa = np.zeros((1,5719))
# np.array(tempa)
tempa = tempa.astype("int")
arr=np.concatenate([arr,np.array(tempa)])
i = arr.shape[0]
user_item_matrix = arr


(432, 5719)


In [134]:
# user_item_matrix.shape
t1 = pd.DataFrame(user_item_matrix,columns=cols)
user_item_matrix = t1

In [89]:
user_item_matrix = user_item_matrix.fillna(0)

In [135]:
def movie_recommender_engine(movie_name, matrix, cf_model, n_recs):
    # Fit model on matrix
    # cf_knn_model.fit(matrix)

    # Extract input movie ID
    movie_id = process.extractOne(movie_name, movie_data['title'])
    sl = int(movie_id[2])
    it = movie_data.iloc[sl]['movieId']
    # matrix[movie_id]

    user_item_matrix.iloc[len(user_item_matrix)-1][it]=2
    # Calculate neighbour distances
    distances, indices = cf_model.kneighbors(np.array(user_item_matrix.iloc[0]).reshape(1,-1), n_neighbors=n_recs)
    movie_rec_ids = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]

    # # List to store recommendations
    cf_recs = []
    for i in movie_rec_ids:
        cf_recs.append({'Title':movie_data['title'][i[0]],'Distance':i[1]})

    # # Select top number of recommendations needed
    df = pd.DataFrame(cf_recs, index = range(1,n_recs))

    return df



In [72]:
def movie_recommender_engine1(movie_name, matrix, cf_model, n_recs):#returns the movie id
    # Fit model on matrix
    cf_knn_model.fit(matrix)

    # Extract input movie ID
    movie_id = process.extractOne(movie_name, movie_data['Jumanji'])[2]
    return movie_id

In [136]:
n_recs = 10
valv = movie_recommender_engine('Tomb Raider', user_item_matrix, cf_knn_model, n_recs)

In [137]:
valv

,Title,Distance
1,Sabrina,0.816234
2,The Secret Adventures of Tom Thumb,0.815674
3,An Awfully Big Adventure,0.815443
4,The Mask,0.809165
5,Waterworld,0.804165
6,Forrest Gump,0.793905
7,Persuasion,0.789529
8,Heavyweights,0.787782
9,Bushwhacked,0.728664


In [128]:
!pip3 list


Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.3
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
beautifulsoup4                   4.12.3
bi